In [2]:
from opensearchpy import OpenSearch
import json
host = 'localhost'
port = 9200
auth = ('admin','admin')



In [3]:
# Create the client with SSL/TLS enabled, but hostname and certification verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    # client_cert = client_cert_path,
    # client_key = client_key_path,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [4]:
#Do a few checks before we start indexing:
print(client.cat.health())
print(client.cat.indices())

1645318248 00:50:48 docker-cluster yellow 1 1 true 8 8 0 0 4 0 - 66.7%

yellow open security-auditlog-2022.02.19                PGiBvPgHTmyI16bdT8uDpw 1 1  817  0 805.6kb 805.6kb
green  open .kibana_92668751_admin_1                    jGcsAkCnQjajrpvfz3yHRw 1 0   38  0  29.4kb  29.4kb
yellow open logstash_test                               e5sraf4ISUyY4NlUOV-psQ 1 1    3  0  16.3kb  16.3kb
yellow open searchml_test                               N9iQ26TlQKWKtqXOfHcvgg 1 1    4  0  21.4kb  21.4kb
green  open .kibana_1                                   wf5MjWZ2TI-JNKzwl1NuDw 1 0    3 11    14kb    14kb
green  open opensearch_dashboards_sample_data_ecommerce TZu84S_MQtutb-sLvcVv4g 1 0 4675  0   4.3mb   4.3mb
green  open .opendistro_security                        rCemxg5jRBiKHSzJSdR_Fg 1 0    9  0  59.9kb  59.9kb
yellow open security-auditlog-2022.02.20                jAmBqCzOR8uGMuFRlNkA1g 1 1   68  0 188.5kb 188.5kb



In [10]:
# If you still have your documents from the Dev Tools test, we should be able to check them here:
print(client.cat.count("searchml_test", params={"v":"true"}))

epoch      timestamp count
1645301091 20:04:51  4



In [8]:
q = 'Hitesh'
index_name = 'logstash_test'
query = {
      'size': 5,
      'query': {
          'multi_match': {
            'query': q,
            'fields': ['message^2', 'host']
            }
          }
        }

client.search(
            body = query,
            index = index_name
        )

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.7563686,
  'hits': [{'_index': 'logstash_test',
    '_type': '_doc',
    '_id': '3zrbE38BhqS8Kq1d43LF',
    '_score': 1.7563686,
    '_source': {'@version': '1',
     'message': 'Hitesh Karunakara Shetty',
     '@timestamp': '2022-02-19T21:20:51.892Z',
     'host': 'hiteshshettyk90-searchwi-soq5n550s5v'}}]}}

In [8]:
q = "*"
index_name = 'bbuy_products'
query = {
      'size': 5,
      'query': {
          'multi_match': {
            'query': q,
            'fields': ['name^2', 'sku']
            }
          }
        }

client.search(
            body = query,
            index = index_name
        )

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [5]:
index_name = 'searchml_test'
query = {
            'size': 5,
            'query': {
                "function_score": {
                    "query": {
                        "bool": {
                            "must": [
                                {"match_all": {}}
                            ],
                            "filter": [
                                {"term": {"category": "childrens"}}
                            ]
                        }
                    },
                    "field_value_factor": {
                        "field": "price",
                        "missing": 1
                    }
                }
            }
        }

client.search(
            body=query,
            index=index_name
        )

{'took': 8,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'searchml_test',
    '_type': '_doc',
    '_id': 'doc_a',
    '_score': 1.0,
    '_source': {'title': 'Fox and Hounds',
     'body': 'The quick red fox jumped over the lazy brown dogs.',
     'category': 'childrens'}},
   {'_index': 'searchml_test',
    '_type': '_doc',
    '_id': 'doc_d',
    '_score': 1.0,
    '_source': {'title': 'The Three Little Pigs Revisted',
     'body': 'The big, bad wolf huffed and puffed and blew the house down. The end.',
     'category': 'childrens'}}]}}

In [7]:
##Aggregations
index_name = 'searchml_test'
query = {
        'size': 0,
        'query': {
            "match_all": {}
            },
            'aggs': {
                "category": {
                    "terms": {
                        "field": "category.keyword",
                        "size": 10,
                        "missing": "N/A",
                        "min_doc_count": 0
                    }
                }
            }
        }

response = client.search(
            body=query,
            index=index_name
        )
print('\nSearch results:')
print(json.dumps(response, indent=4))


Search results:
{
    "took": 32,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 5000,
            "relation": "eq"
        },
        "max_score": null,
        "hits": []
    },
    "aggregations": {
        "category": {
            "doc_count_error_upper_bound": 0,
            "sum_other_doc_count": 0,
            "buckets": [
                {
                    "key": "N/A",
                    "doc_count": 5000
                }
            ]
        }
    }
}


In [6]:
  index_name = 'opensearch_dashboards_sample_data_ecommerce'
  query = {
  'size': 0,
  'query': {
   "match_all": {}
   },
   'aggs': {
    "price": {
     "terms": {
      "field": "price",
      "size": 10,
      "min_doc_count": 0
     }
    }
   }
  }

  response = client.search(
   body=query,
   index=index_name
  )
  print('\nSearch results:')
  print(json.dumps(response, indent=4))


Search results:
{
    "took": 11,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 5000,
            "relation": "eq"
        },
        "max_score": null,
        "hits": []
    },
    "aggregations": {
        "price": {
            "doc_count_error_upper_bound": 0,
            "sum_other_doc_count": 0,
            "buckets": []
        }
    }
}


In [5]:
client = OpenSearch(
            hosts=[{'host': host, 'port': port}],
            http_compress=True,  # enables gzip compression for request bodies
            http_auth=auth,
            # client_cert = client_cert_path,
            # client_key = client_key_path,
            use_ssl=True,
            verify_certs=False, # set to true if you have certs
            ssl_assert_hostname=False,
            ssl_show_warn=False,

        )
        
query = {
        "size": 5,
        "query": {
            "match_all": {} # Replace me with a query that both searches and filters
        },
        "aggs": {
            #TODO: FILL ME IN
        }
}
response  = client.search(body=query, index="bbuy_products") 
print(response)


{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'bbuy_products', '_type': '_doc', '_id': '1613301', '_score': 1.0, '_source': {'class': ['BLU RAY MOVIES'], 'inStorePickup': ['false'], 'categoryPath': ['Best Buy', 'Movies &amp; Music', 'Movies &amp; TV Shows'], 'productId': ['50156'], 'onlineAvailability': ['false'], 'regularPrice': ['9.99'], 'quantityLimit': ['3'], 'digital': ['false'], 'name': ['The Frugal Gourmet: Whole-Meal Soups - VHS'], 'type': ['Movie'], 'releaseDate': ['2001-05-15'], '@version': '1', 'bestBuyItemId': ['233422'], 'salePrice': ['9.99'], 'categoryPathIds': ['cat00000', 'abcat0600000', 'cat02015'], '@timestamp': '2022-02-21T06:05:28.350Z', 'tags': ['multiline'], 'department': ['VIDEO/COMPACT DISC'], 'subclassId': ['1029'], 'categoryLeaf': ['cat02015'], 'categoryPathCount': ['3.0'], 'departmentId': ['8'], 'homeDelivery': ['